# Aligns GPR, TRCs and Drainage Information

In [1]:
# Imports
import pandas as pd
import numpy as np
import statistics as stats


In [2]:
# Global Variables
metrageInc = 5 # 5 metre increments
numTRCRuns = 4 # Number of runs of TRC
TRCSkipRows = 4 # Number of rows to skip at top of TRC CSV
GPRSkipRows = 14 # Number of rows to skip at top of GPR XLS
drainageSkipRows = 0 # Number of rows to skip at top of drainage XLS

# File Parameters
dir_loc = './Data/'

GPRFileName = dir_loc + '2018 GPR SF 401.xlsx'

TRC_file1 = dir_loc + '531401 - DM A 0-9.69 NORTHGATE-SANDGATE-201701311116.csv'
TRC_file2 = dir_loc + '531401 - DM A 0-9.69 NORTHGATE-SANDGATE-201707041300.csv'
TRC_file3 = dir_loc + '531401 - DM A 0-9.69 NORTHGATE-SANDGATE-201710171136.csv'
TRC_file4 = dir_loc + '531401 - DM A 0-9.69 NORTHGATE-SANDGATE-201801311009.csv'

drainage_file = dir_loc + 'Drainage locations C139.xlsx'

### Read GPR Data

In [3]:
#### GPR
GPRSheetName = 'Sheet1'
division = 'Brisbane'
divisionCol = 'Division' # column name of division
subdivision = 'SF' # Shorncliffe line
subdivisionCol = 'Sub-division' # column name of subdivision
#lineSegment = '531'
#lineSegmentCol = 2 # column name of line segment
track = 401 # Up or down line
trackCol = 'Track ID' # column name of track direction
StartKM = "Start KM" # Name of Start KM column
EndKM = "End KM" # Name of Start KM column

# Establish output file
outFile = dir_loc + subdivision + str(track) + 'OUT_Nov19.xlsx' # Output File

In [4]:
# Read GPR 
GPRDF = pd.read_excel(GPRFileName, sheet_name = GPRSheetName, skiprows=GPRSkipRows)

# Filter GPR DF by segment etc
GPRSegment = GPRDF.loc[ (GPRDF[divisionCol]==division) & (GPRDF[subdivisionCol]==subdivision ) & 
                       (GPRDF[trackCol]==track )]

GPRSegment.loc[0:2,:]

,Unnamed: 0,Division,Sub-division,Line Segment,Track ID,Collection Date,Start KM,End KM,Geolink,Dec.Lat,...,Category.6,Category.7,Category.8,Unnamed: 46,Left*.4,Right*.4,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52
0,NaN,Brisbane,SF,531,401,2018-10-02,0.00342,0.00839,geolink,-27.392896,...,1,1,1,NaN,3,1,NaN,NaN,0,NaN
1,NaN,Brisbane,SF,531,401,2018-10-02,0.00839,0.01335,geolink,-27.392851,...,1,1,1,NaN,3,1,NaN,NaN,0,NaN
2,NaN,Brisbane,SF,531,401,2018-10-02,0.01335,0.01832,geolink,-27.392806,...,1,1,1,NaN,3,1,NaN,NaN,1,NaN


In [5]:
# GPR Features
PVCLeft = 'PVC Value' # PVC columns
PVCCentre = 'PVC Value.1' 
PVCRight = 'PVC Value.2' 

LRILeft = 'Left*' # LRI columns
LRICentre = 'Centre'
LRIRight = 'Right*'

BTILeft = 'Left*.1' # BTI columns
BTICentre = 'Centre.1'
BTIRight = 'Right*.1'

MLILeft = 'Left*.2' # MLI columns
MLICentre = 'Centre.2'
MLIRight = 'Right*.2'

## Not included in 2018
# SMLILeft = 'Left*.3' # SMLI columns
# SMLICentre = 'Centre.3'
# SMLIRight = 'Right.3'

FDLLeft = 'Left*.3' # FDL columns
FDLCentre = 'Centre.3'
FDLRight = 'Right*.3'

## Not included in 2018
# CTQILeft = 'Left*.5' # CTQI columns
# CTQICentre = 'Centre.5'
# CTQIRight = 'Right*'

BVMLeftVolume = 'Volume (cubic m)' # BVM columns
BVMLeftCategory = 'Category.3'
BVMCentreVolume = 'Volume (cubic m).1' 
BVMCentreCategory = 'Category.4'
BVMRightVolume = 'Volume (cubic m).2' 
BVMRightCategory = 'Category.5'

# New for 2018
BDMLeft = 'Category.6' # BDM columns
BDMCentre = 'Category.7'
BDMRight = 'Category.8'

# New for 2018
TDILeft = 'Left*.4' # TDI columns
TDIRight = 'Right*.4'

SMI = 'Unnamed: 51' # SMI column


### Read TRC Data

In [6]:
#### TRC
TRCTopL = 'TOP L' # Name of Top Left column
TRCTopR = 'TOP R' # Name of Top Right column
TRCTwist3 = 'TW 3' # Name of Twist 3 column
TRCTwist10 = 'TW 10' # Name of Twist 10 column
TRCVersL = 'VER L' # Name of Versine Left column
TRCVersR = 'VER R' # Name of Versine Right column


In [7]:
TRC_DF1 = pd.read_csv(TRC_file1, skiprows=TRCSkipRows) 
TRC_DF1.rename(columns=lambda x: x.strip(), inplace=True)
TRC_DF1.loc[0:2,:]

,METRAGE,GAUGE,SUPER,TOP L,TOP R,TW 10,TW 3,VER L,VER R,CON F,GR HT,VOLT,TR ST,OV HT,AC LN,AC VT,
0,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.1,0.0,-291.5,0.3,0.1,NaN
2,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3,0.0,-289.6,0.3,0.1,NaN


In [11]:
TRC_DF2 = pd.read_csv(TRC_file2, skiprows=TRCSkipRows) 
TRC_DF2.rename(columns=lambda x: x.strip(), inplace=True)
TRC_DF2.loc[0:2,:]

,METRAGE,GAUGE,SUPER,TOP L,TOP R,TW 10,TW 3,VER L,VER R,CON F,GR HT,VOLT,TR ST,OV HT,AC LN,AC VT,
0,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.4,0.0,0.0,0.0,0.1,NaN
1,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3,0.0,-351.9,0.3,0.3,NaN
2,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.4,0.0,-350.9,0.3,0.1,NaN


In [12]:
TRC_DF3 = pd.read_csv(TRC_file3, skiprows=TRCSkipRows) 
TRC_DF3.rename(columns=lambda x: x.strip(), inplace=True)
TRC_DF3.loc[0:2,:]

,METRAGE,GAUGE,SUPER,TOP L,TOP R,TW 10,TW 3,VER L,VER R,CON F,GR HT,VOLT,TR ST,OV HT,AC LN,AC VT,
0,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,-327.4,0.6,0.1,NaN
2,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,-323.6,0.1,0.1,NaN


In [13]:
TRC_DF4 = pd.read_csv(TRC_file4, skiprows=TRCSkipRows) 
TRC_DF4.rename(columns=lambda x: x.strip(), inplace=True)
TRC_DF4.loc[0:2,:]

,METRAGE,GAUGE,SUPER,TOP L,TOP R,TW 10,TW 3,VER L,VER R,CON F,GR HT,VOLT,TR ST,OV HT,AC LN,AC VT,
0,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.8,0.0,-338.0,0.1,0.1,NaN
2,0.0,1067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.8,0.0,-339.9,0.3,0.3,NaN


In [93]:
# Align TRC datasets
def align_TRC(df1, df2): # Align df2 with df1
    nsamples = 100
    measures = ['GAUGE', 'SUPER']
    meas_min = dict()

    for m in measures:
        #print('Looking at measure: {0}'.format(m))
        counts = []
        temp_df = pd.concat([pd.DataFrame(df1.loc[:,m]), pd.DataFrame(df2.loc[:,m])], axis=1)
        #print(temp_df.shape)
        for i in range(nsamples):
            val1 = temp_df.iloc[500:-500, 0].values
            val2 = temp_df.iloc[500-int(nsamples/2)+i:-500-int(nsamples/2)+i, 1].values
            diff1 = val1 - val2
            ans = np.std(diff1[~np.isnan(diff1)])
            counts.append(ans)
        min_std = np.argmin(np.array(counts))
        #print('min_std=',min_std)
        meas_min[m]=min_std-50
    g = list(meas_min.values())

    #print('g=',g)
    rec = round(sum(g)/len(g)) # return the mode of alignment
    print('recommended: shift TRC_DF by {0} compared to last TRC DF'.format(rec))
    return rec

In [124]:
# Establish a list of TRC dataframes
TRCList = []
TRCList.append(TRC_DF1)
TRCList.append(TRC_DF2)
TRCList.append(TRC_DF3)
TRCList.append(TRC_DF4) # Most recent run, used as response 

# The last TRC_DF will be used to align all other TRCs
for i in range(numTRCRuns-1):
    shift_val = int(align_TRC(TRCList[numTRCRuns-1], TRCList[i]))
    TRCList[i] = TRCList[i].shift(shift_val) # Shift the TRC_DF against the last
    TRCList[i]['METRAGE'] = TRCList[numTRCRuns-1]['METRAGE'] # Adopt the metrage used by last TRC_DF


recommended: shift TRC_DF by 13.0 compared to last TRC DF
recommended: shift TRC_DF by 16.0 compared to last TRC DF
recommended: shift TRC_DF by -2.0 compared to last TRC DF


In [126]:
startMetrage = 0.0025 # Starting metrage of TRC measurement
calcDF = pd.DataFrame(columns=['METRAGE', 'Start Km', 'End Km','PVCLeft', 'PVCCentre', 'PVCRight',
                               'LRILeft', 'LRICentre', 'LRIRight', 'BTILeft', 'BTICentre', 'BTIRight',
                               'MLILeft', 'MLICentre', 'MLIRight', 'FDLLeft', 'FDLCentre', 'FDLRight', 
                               'BVMLeftVolume', 'BVMLeftCategory', 'BVMCentreVolume', 'BVMCentreCategory',
                               'BVMRightVolume', 'BVMRightCategory', 'BDMLeft', 'BDMCentre', 'BDMRight',
                               'TDILeft', 'TDIRight','SMI', 'Drainage',
                               'SDTopLeft1', 'SDTopLeft2','SDTopLeft3','SDTopLeft4',
                               'SDTopRight1','SDTopRight2','SDTopRight3','SDTopRight4',
                               'SDTwist101','SDTwist102','SDTwist103','SDTwist104',
                               'SDTwist31','SDTwist32','SDTwist33','SDTwist34',
                               'SDVersL1', 'SDVersL2', 'SDVersL3', 'SDVersL4',
                               'SDVersR1', 'SDVersR2', 'SDVersR3', 'SDVersR4',
                               'Combined1','Combined2','Combined3','Combined4'])

# Find largest distance measured by final TRC run
maxDist = float(TRCList[numTRCRuns-1].iloc[-1:,]['METRAGE'])
# maxDist = np.min( [float(TRCList[0].iloc[-1:,]['METRAGE']), float(TRCList[1].iloc[-1:,]['METRAGE']), 
#     float(TRCList[2].iloc[-1:,]['METRAGE']), float(TRCList[3].iloc[-1:,]['METRAGE']) ])


for i in range( int(maxDist*1000/metrageInc) ):
    calcDF.loc[i, 'METRAGE'] = startMetrage + i*metrageInc/1000
    calcDF.loc[i, 'Start Km'] = calcDF.loc[i, 'METRAGE'] - metrageInc/1000/2
    calcDF.loc[i, 'End Km'] = calcDF.loc[i, 'METRAGE'] + metrageInc/1000/2


In [127]:
# Calculate standard deviations: Top Left, Top Right and Twist for 4 TRC Runs
colLeft = 'SDTopLeft'
colRight = 'SDTopRight'
colTwist10 = 'SDTwist10'
colTwist3 = 'SDTwist3'
colVersLeft = 'SDVersL'
colVersRight = 'SDVersR'
colCombined = 'Combined'

for j  in range(0, numTRCRuns):
        colNameLeft = colLeft + str(j+1)
        colNameRight = colRight + str(j+1)
        colNameTwist10 = colTwist10 + str(j+1)
        colNameTwist3 = colTwist3 + str(j+1)
        colNameVersLeft = colVersLeft + str(j+1)
        colNameVersRight = colVersRight + str(j+1)
        colNameCombined = colCombined + str(j+1)
        
        for i in range(2, len(calcDF)-2):
            inRange = np.where( (TRCList[j].loc[:,'METRAGE'] > calcDF.loc[i-2,'METRAGE']) & 
                                                    (TRCList[j].loc[:, 'METRAGE'] <= calcDF.loc[i+2, 'METRAGE']) ) 
            calcDF.loc[i,colNameLeft] = stats.stdev( TRCList[j].loc[inRange][TRCTopL] )
            calcDF.loc[i,colNameRight] = stats.stdev( TRCList[j].loc[inRange][TRCTopR] )
            calcDF.loc[i,colNameTwist10] = stats.stdev( TRCList[j].loc[inRange][TRCTwist10] )
            calcDF.loc[i,colNameTwist3] = stats.stdev( TRCList[j].loc[inRange][TRCTwist3] )
            calcDF.loc[i,colNameVersLeft] = stats.stdev( TRCList[j].loc[inRange][TRCVersL] )
            calcDF.loc[i,colNameVersRight] = stats.stdev( TRCList[j].loc[inRange][TRCVersR] )

            # Original QR combined calculation
            calcDF.loc[i,colNameCombined] =  (calcDF.loc[i,colNameLeft] + calcDF.loc[i,colNameRight])/2 + calcDF.loc[i,colNameTwist3]

            # Modified combined calculation
            # (SDL + SDR)/2 + (SDTW3+SDTW10)/2 + (SDVersL+SDVersR)/2
#             calcDF.loc[i,colNameCombined] =  (calcDF.loc[i,colNameLeft] + calcDF.loc[i,colNameRight])/2 + 
#             (calcDF.loc[i,colNameTwist3] + calcDF.loc[i,colNameTwist10])/2 +
#             (calcDF.loc[i,colNameVersLeft] + calcDF.loc[i,colNameVersRight])/2
            
        calcDF.loc[1,colNameLeft] = calcDF.loc[2, colNameLeft]
        calcDF.loc[1,colNameRight] = calcDF.loc[2, colNameRight]
        calcDF.loc[1,colNameTwist10] = calcDF.loc[2, colNameTwist10]
        calcDF.loc[1,colNameTwist10] = calcDF.loc[2, colNameTwist10]
        calcDF.loc[1,colNameVersLeft] = calcDF.loc[2, colNameVersLeft]
        calcDF.loc[1,colNameVersRight] = calcDF.loc[2, colNameVersRight]
        calcDF.loc[1,colNameCombined] = calcDF.loc[2, colNameCombined]
  

### Merge GPR Data

In [129]:
# Insert GPR data
for i in range(1, len(calcDF)-2):
    inRange = np.where( (round(GPRSegment.loc[:, StartKM],3) <= round((calcDF.loc[i,'METRAGE']),3))  &
               (round(GPRSegment.loc[:, EndKM],3) >= round((calcDF.loc[i,'METRAGE']),3)))
    
    # Only process if found unique corresponding range in GPR
    if (len(inRange[0]) == 1):
        calcDF.loc[i,'PVCLeft'] = GPRSegment.iloc[inRange[0][0]][PVCLeft]   
        calcDF.loc[i,'PVCCentre'] = GPRSegment.iloc[inRange[0][0]][PVCCentre]
        calcDF.loc[i, 'PVCRight'] = GPRSegment.iloc[inRange[0][0]][PVCRight]
        
        calcDF.loc[i,'LRILeft'] = GPRSegment.iloc[inRange[0][0]][LRILeft]   
        calcDF.loc[i,'LRICentre'] = GPRSegment.iloc[inRange[0][0]][LRICentre]
        calcDF.loc[i, 'LRIRight'] = GPRSegment.iloc[inRange[0][0]][LRIRight]
        
        calcDF.loc[i,'BTILeft'] = GPRSegment.iloc[inRange[0][0]][BTILeft]   
        calcDF.loc[i,'BTICentre'] = GPRSegment.iloc[inRange[0][0]][BTICentre]
        calcDF.loc[i, 'BTIRight'] = GPRSegment.iloc[inRange[0][0]][BTIRight]
        
        calcDF.loc[i,'MLILeft'] = GPRSegment.iloc[inRange[0][0]][MLILeft]   
        calcDF.loc[i,'MLICentre'] = GPRSegment.iloc[inRange[0][0]][MLICentre]
        calcDF.loc[i, 'MLIRight'] = GPRSegment.iloc[inRange[0][0]][MLIRight]

        calcDF.loc[i,'FDLLeft'] = GPRSegment.iloc[inRange[0][0]][FDLLeft]   
        calcDF.loc[i,'FDLCentre'] = GPRSegment.iloc[inRange[0][0]][FDLCentre]
        calcDF.loc[i, 'FDLRight'] = GPRSegment.iloc[inRange[0][0]][FDLRight]

        calcDF.loc[i,'BVMLeftVolume'] = GPRSegment.iloc[inRange[0][0]][BVMLeftVolume]
        calcDF.loc[i,'BVMLeftCategory'] = GPRSegment.iloc[inRange[0][0]][BVMLeftCategory]
        calcDF.loc[i,'BVMCentreVolume'] = GPRSegment.iloc[inRange[0][0]][BVMCentreVolume]
        calcDF.loc[i,'BVMCentreCategory'] = GPRSegment.iloc[inRange[0][0]][BVMCentreCategory]
        calcDF.loc[i,'BVMRightVolume'] = GPRSegment.iloc[inRange[0][0]][BVMRightVolume]
        calcDF.loc[i,'BVMRightCategory'] = GPRSegment.iloc[inRange[0][0]][BVMRightCategory]

        calcDF.loc[i,'BDMLeft'] = GPRSegment.iloc[inRange[0][0]][BDMLeft]   
        calcDF.loc[i,'BDMCentre'] = GPRSegment.iloc[inRange[0][0]][BDMCentre]
        calcDF.loc[i, 'BDMRight'] = GPRSegment.iloc[inRange[0][0]][BDMRight]

        calcDF.loc[i,'TDILeft'] = GPRSegment.iloc[inRange[0][0]][TDILeft]   
        calcDF.loc[i,'TDIRight'] = GPRSegment.iloc[inRange[0][0]][TDIRight]
        
        calcDF.loc[i,'SMI'] = GPRSegment.iloc[inRange[0][0]][SMI]


### Read and merge drainage points

In [130]:
# Insert drainage points
drainageDF = pd.read_excel(drainage_file, skiprows=drainageSkipRows)

# Initialise to no drainage points
calcDF['Drainage'] = 0

# Insert "1" where find a drainage point
for i in range(len(drainageDF)):
    inRange = list(np.where( (calcDF['Start Km'] <= drainageDF.loc[i, 'Start Point']) & # Drainage start point
                       (calcDF['End Km'] >= drainageDF.loc[i, 'Start Point'])))

    inRange.append(list(np.where( (calcDF['Start Km'] <= drainageDF.loc[i, 'End Point']) & # Drainage start point
                       (calcDF['End Km'] >= drainageDF.loc[i, 'End Point']))))

    # Insert "1" for all drainage point indices in inRange
    inRange = list(inRange[0])
    for j in (inRange):
        calcDF.loc[j,'Drainage'] = 1   


In [131]:
export_excel = calcDF.to_excel (outFile, index = None, header=True)
